Primeiro passo importando as bibliotecas

In [1]:
import pandas as pd
import csv #importando csv


- A pesquisa ficou adestrita a região do vale do Jauru

- Região vale do Jauru

        Araputanga
        Figueirópolis d'Oeste
        Glória d'Oeste
        Indiavaí
        Jauru
        Lambari d'Oeste
        Mirassol d'Oeste
        Porto Esperidião
        Reserva do Cabaçal
        Rio Branco
        Salto do Céu
        São José dos Quatro Marcos


In [2]:

cidade_link = open('src/arquivos.csv')

implementa = csv.DictReader(cidade_link)
#a leitura só funcionara com os parametros abaixo com o parametro acimas

for inplemente in implementa:
    #lendo arquivo python para rastrear o separador
    carregar_separador = open(inplemente['link'],encoding='ISO-8859-1')
    separa = csv.Sniffer().sniff(carregar_separador.read())
    separador = separa.delimiter
    print("Tipo de separador definido foi --> ", separador)
    #---------------------------------------
    #dividindo municipio para colocar na variavel de nome    
    dividi_municipio = inplemente['municipio'].split(' ')# o codigo dividi o nome do municipio
    #----------------------------------------------
    
    # Carregando csv no pandas 
    servidores = pd.read_csv(inplemente['link']#chama o arquivo csv 
                             ,sep=separador #informa o separador do arquivo
                             ,encoding='ISO-8859-1', #informa a codificação do arquivo
                             error_bad_lines=False,
                             engine='python') #trata os erros nas linhas
    nome_variavel = 'servidores_{}'.format(dividi_municipio[0])     #alterando o nome da variavel
    globals()[nome_variavel] = servidores
    globals()[nome_variavel]['Municipio'] = inplemente['municipio']
    print(nome_variavel)
    print('-------------------------')



Tipo de separador definido foi -->  ,
servidores_Figueirópolis
-------------------------
Tipo de separador definido foi -->  ,
servidores_Figueirópolis
-------------------------
Tipo de separador definido foi -->  ;
servidores_Indiavaí
-------------------------
Tipo de separador definido foi -->  ;
servidores_Indiavaí
-------------------------
Tipo de separador definido foi -->  ;
servidores_Jauru
-------------------------
Tipo de separador definido foi -->  ;
servidores_Jauru
-------------------------
Tipo de separador definido foi -->  ;
servidores_Lambari
-------------------------
Tipo de separador definido foi -->  ;
servidores_Lambari
-------------------------
Tipo de separador definido foi -->  ;
servidores_Mirassol
-------------------------
Tipo de separador definido foi -->  ;
servidores_Mirassol
-------------------------
Tipo de separador definido foi -->  ;
servidores_Porto
-------------------------
Tipo de separador definido foi -->  ;
servidores_Porto
----------------------

- foi necessario criar uma variavel com todos os servidores dos municipios da região oeste

In [3]:
servidores_municipio_região_oeste = servidores_Lambari, servidores_Indiavaí, servidores_Jauru, servidores_Mirassol, servidores_Porto, servidores_Reserva, servidores_São, servidores_Figueirópolis,servidores_Salto, servidores_Rio, servidores_Araputanga

In [4]:
resultado = pd.concat(servidores_municipio_região_oeste)
resultado.to_csv('servidores_selecionado.csv')

In [12]:
resultado

,Detalhe,Referência,Matrícula,Data Desligamento,Tipo de Contrato,Vínculo,Nome,Cargo,Cargo inicial,Proventos,Descontos,Líquido,Divisão,Subdivisão,Unidade,Categoria Funcional,Data Prevista Termino Contrato,Municipio,Data Admissão
0,NaN,Folha Mensal - Maio,195.0,NaN,Efetivo,Servidor Público Efetivo,ADIR DE SOUZA LEITE,FISCAL SANITARIO,FISCAL SANITARIO,"2.385,40","279,31","2.106,09",SECRETARIA MUNICIPAL DE SAUDE,FUNDO MUNICIPAL DE SAUDE,MAUNT.ENC.FUNDO MUNICIPAL E SAUDE - EFETIVO,EFETIVO,NaN,Lambari d'Oeste,NaN
1,NaN,Folha Mensal - Maio,1810.0,NaN,Efetivo,Servidor Público Efetivo,ADRIANA APARECIDA LUIZ SEGANTINI,TECNICO EM ENFERMAGEM,TECNICO EM ENFERMAGEM,"3.215,66","708,42","2.507,24",SECRETARIA MUNICIPAL DE SAUDE,FUNDO MUNICIPAL DE SAUDE,MAUNT.ENC.FUNDO MUNICIPAL E SAUDE - EFETIVO,EFETIVO,NaN,Lambari d'Oeste,NaN
2,NaN,Folha Mensal - Maio,2.0,NaN,Efetivo,Servidor Público Efetivo,AILTON ALMEIDA PEREIRA,OPERADOR DE MAQUINAS,OPERADOR DE MAQUINAS,"2.956,45","409,31","2.547,14",SEC. MUN. DE INFRA ESTRUTURA,DPTO DE VIACAO E OBRAS,MANUT.ENC. SEC MUN. DE INFRA ESTRUTURA-EFETIVO,EFETIVO,NaN,Lambari d'Oeste,NaN
3,NaN,Folha Mensal - Maio,267.0,NaN,Contrato Temporário,Contrato Prazo Determ. (RJU),ALDAIR DIAS MARTINS,AGENTE COMUNITARIO DE SAUDE,AGENTE COMUNITARIO DE SAUDE,"1.600,00","134,97","1.465,03",SECRETARIA MUNICIPAL DE SAUDE,FUNDO MUNICIPAL DE SAUDE,MAN.PROG. AGENTES COMUNIT DE SAUDE-PACS - CONT...,CONTRATADO,NaN,Lambari d'Oeste,NaN
4,NaN,Folha Mensal - Maio,2318.0,NaN,Contrato Temporário,Contrato Prazo Determ. (RJU),ALINE BATISTA DE SOUZA,PROFESSOR NIVEL SUPERIOR-06 H,PROFESSOR NIVEL SUPERIOR-06 H,"772,58","57,94","714,64",SEC. MUN. DE EDUCACAO E CULTURA,FUNDEB,MANUT. FUNDEB 60% CONTRATADO,CONTRATADO,NaN,Lambari d'Oeste,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,NaN,Folha Mensal - Maio,33680.0,NaN,Efetivo em Comissão,SERVIDOR REGIDO PELO REGIME JURÍDICO ÚNICO (FE...,WILLIE DOUGLAS MARTES FERREIRA,GERENTE DE INDUSTRIA E COMERCIO,TECNICO FLORESTAL,"5.241,61","1.824,48","3.417,13",SECRETARIA DE MEIO AMBI E DES URBANO RUR,112 SECRETARIA DE MEIO AMBI E DES URBANO RUR,DEMAIS SERVIDORES - SECRETARIA DE MEIO AMBIENTE,NaN,NaN,Araputanga,NaN
477,NaN,Folha Mensal - Maio,20565.0,NaN,Efetivo,SERVIDOR REGIDO PELO REGIME JURÍDICO ÚNICO (FE...,ZELIA BARBOSA GONCALVES,PROFESSOR,PROFESSOR,"3.288,06","1.610,28","1.677,78",SECRETARIA DE EDUCACAO E CULTURA,SECRETARIA DE EDUCACAO E CULTURA,FUNDEB PROF. 1 A 8 60% - 200-0.1.18,NaN,NaN,Araputanga,NaN
478,NaN,Folha Mensal - Maio,20759.0,NaN,Efetivo,SERVIDOR REGIDO PELO REGIME JURÍDICO ÚNICO (FE...,ZELIA PEREIRA REBOLI,PROFESSOR,PROFESSOR,"5.154,20","3.327,48","1.826,72",SECRETARIA DE EDUCACAO E CULTURA,SECRETARIA DE EDUCACAO E CULTURA,EDUCACAO INFANTIL 3/3 60,NaN,NaN,Araputanga,NaN
479,NaN,Folha Mensal - Maio,21536.0,NaN,Efetivo,SERVIDOR REGIDO PELO REGIME JURÍDICO ÚNICO (FE...,ZENAIDE APARECIDA DOS SANTOS MORAES,OPERADOR DO SMAE,OPERADOR DO SMAE,"1.779,93","848,86","931,07",SECRETARIA DE OBRAS E INFRAESTRUTURA,136 SECRETARIA DE OBRAS E INFRAESTRUTURA,DEMAIS SERVIDORES - SMAE,NaN,NaN,Araputanga,NaN


#### FONTE DE DADOS
- SITE BRASIL IO 
* https://brasil.io/dataset/govbr/auxilio_emergencial/

#### METODO
- baixar os dados por municipios e salvar
- **DIFICULDADES**
    - O ARQUIVO VINHA COM MASCARA ENTÃO ORDENOU POR NUMERO PARA IMPLEMENTAR UMA BUSCA COM O FOR 

Implementando codigo para unir dados baixados do site brasil io

In [6]:
for i in range(1,12):
    print('df'+str(i))
    df = pd.read_csv("src/dados_brasil_io/govbr-{}.csv".format(i))
    nome_variavel = 'df{}'.format(i)
    globals()[nome_variavel] = df

df1
df2
df3
df4
df5
df6
df7
df8
df9
df10
df11


Unindo os dataframes

In [7]:
auxilio_emergencial = df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 ,df11
auxilio_emergencial_vale_jauru = pd.concat(auxilio_emergencial)

In [8]:
auxilio_emergencial_vale_jauru

,ano_mes,uf,municipio,nis_beneficiario,cpf_beneficiario,beneficiario,nis_responsavel,cpf_responsavel,responsavel,enquadramento,parcela,valor
0,202005,MT,Indiavaí,NaN,***200171**,ADELSON GOMES SILVA,-2,NaN,NaN,EXTRA CADUN,1,600.0
1,202005,MT,Indiavaí,NaN,***397728**,ALINE DIAS MAGALHAES,-2,NaN,NaN,EXTRA CADUN,1,600.0
2,202005,MT,Indiavaí,NaN,***996641**,ANA PAULA DA SILVA PRATES,-2,NaN,NaN,EXTRA CADUN,1,600.0
3,202005,MT,Indiavaí,NaN,***040361**,ANDERSON AUGUSTO DOS SANTOS,-2,NaN,NaN,EXTRA CADUN,1,600.0
4,202005,MT,Indiavaí,NaN,***887201**,ANDREIA PIRES BORGES,-2,NaN,NaN,EXTRA CADUN,1,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1295,202004,MT,Rio Branco,NaN,***923871**,ZENAIDE VITTORAZZI RIOS,-2,NaN,NaN,EXTRA CADUN,1,1200.0
1296,202004,MT,Rio Branco,2.041920e+10,***364651**,ZENILDA MARQUES DE OLIVEIRA,20419196956,***364651**,ZENILDA MARQUES DE OLIVEIRA,BOLSA FAMILIA,1,600.0
1297,202004,MT,Rio Branco,2.041919e+10,***969521**,ZILDA ANDRADE DE ASSIS,20419186373,***969521**,ZILDA ANDRADE DE ASSIS,BOLSA FAMILIA,1,1200.0
1298,202004,MT,Rio Branco,2.092871e+10,***295061**,ZILDA SANTOS DE OLIVEIRA,-2,NaN,NaN,CADUN NAO BOLSA,1,600.0


In [13]:
auxilio_emergencial_vale_jauru = auxilio_emergencial_vale_jauru.rename(columns={'beneficiario':'Nome'})
auxilio_emergencial_vale_jauru.head()

,ano_mes,uf,municipio,nis_beneficiario,cpf_beneficiario,Nome,nis_responsavel,cpf_responsavel,responsavel,enquadramento,parcela,valor
0,202005,MT,Indiavaí,NaN,***200171**,ADELSON GOMES SILVA,-2,NaN,NaN,EXTRA CADUN,1,600.0
1,202005,MT,Indiavaí,NaN,***397728**,ALINE DIAS MAGALHAES,-2,NaN,NaN,EXTRA CADUN,1,600.0
2,202005,MT,Indiavaí,NaN,***996641**,ANA PAULA DA SILVA PRATES,-2,NaN,NaN,EXTRA CADUN,1,600.0
3,202005,MT,Indiavaí,NaN,***040361**,ANDERSON AUGUSTO DOS SANTOS,-2,NaN,NaN,EXTRA CADUN,1,600.0
4,202005,MT,Indiavaí,NaN,***887201**,ANDREIA PIRES BORGES,-2,NaN,NaN,EXTRA CADUN,1,1200.0


In [14]:
ser_rec_aux = auxilio_emergencial_vale_jauru.merge(resultado, on='Nome')

In [15]:
ser_rec_aux

,ano_mes,uf,municipio,nis_beneficiario,cpf_beneficiario,Nome,nis_responsavel,cpf_responsavel,responsavel,enquadramento,...,Proventos,Descontos,Líquido,Divisão,Subdivisão,Unidade,Categoria Funcional,Data Prevista Termino Contrato,Municipio,Data Admissão
0,202005,MT,Indiavaí,NaN,***200171**,ADELSON GOMES SILVA,-2,NaN,NaN,EXTRA CADUN,...,"1.917,57","645,66","1.271,91",SECRETARIA ASSISTENCIA DESENV. SOCIAL,GERENCIA ESPORTE LAZER,DEPARTAMENTO DE ESPORTE E LAZER - COMISSIONADOS,NaN,NaN,Indiavaí,04/09/2017
1,202004,MT,Indiavaí,1.251689e+10,***193421**,EDILSON SOARES DE SOUZA,-2,NaN,NaN,CADUN NAO BOLSA,...,"1.597,98","128,13","1.469,85",SECRETARIA ASSISTENCIA DESENV. SOCIAL,GERENCIA CULTURA TURISMO,FUNDO MUN. ASSISTENCIA SOCIAL - COMISSIONADOS,NaN,NaN,Indiavaí,06/04/2020
2,202004,MT,Indiavaí,NaN,***692341**,JOSE CARLOS DA SILVA,-2,NaN,NaN,EXTRA CADUN,...,"1.347,80","123,08","1.224,72",SECRETARIA MUNICIPAL DE ADMINISTRAÇÃO E PLANEJ...,DEPARTAMENTO DE ADMINISTRAÇÃO GERAL,DEPARTAMENTO DE ADMINISTRAÇÃO GERAL EFETIVOS,FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,NaN,Salto do Céu,NaN
3,202004,MT,São José dos Quatro Marcos,1.280754e+10,***553991**,JOSE CARLOS DA SILVA,16410444487,***597361**,MARIA DE FATIMA ANTUNES DA SILVA,BOLSA FAMILIA,...,"1.347,80","123,08","1.224,72",SECRETARIA MUNICIPAL DE ADMINISTRAÇÃO E PLANEJ...,DEPARTAMENTO DE ADMINISTRAÇÃO GERAL,DEPARTAMENTO DE ADMINISTRAÇÃO GERAL EFETIVOS,FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,NaN,Salto do Céu,NaN
4,202004,MT,São José dos Quatro Marcos,2.033952e+10,***791901**,JOSE CARLOS DA SILVA,20339524833,***791901**,JOSE CARLOS DA SILVA,BOLSA FAMILIA,...,"1.347,80","123,08","1.224,72",SECRETARIA MUNICIPAL DE ADMINISTRAÇÃO E PLANEJ...,DEPARTAMENTO DE ADMINISTRAÇÃO GERAL,DEPARTAMENTO DE ADMINISTRAÇÃO GERAL EFETIVOS,FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,NaN,Salto do Céu,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,202004,MT,Rio Branco,NaN,***473311**,ANTONIO PEREIRA DA SILVA,-2,NaN,NaN,EXTRA CADUN,...,"2.310,56","519,37","1.791,19",SECRETARIA MUNICIPAL DE SAUDE,FUNDO MUN. SAUDE,GABINETE DA SECR. DE SAUDE - EFETIVO,EFETIVO,NaN,Porto Esperidião,11/11/2009
252,202004,MT,Rio Branco,NaN,***753831**,ELIANYS PETERLE,-2,NaN,NaN,EXTRA CADUN,...,"3.299,39","372,99","2.926,40",SECRETARIA MUNICIPAL DE FINAN?AS,DEPARTAMENTO DE FINAN?AS,MANUT.ENC. DEPARTAMENTO DE FINANÇAS - COMISSIO...,COMISSIONADO,NaN,Lambari d'Oeste,NaN
253,202004,MT,Rio Branco,1.636643e+10,***980331**,LEILIANE SILVA DE SOUZA,-2,NaN,NaN,CADUN NAO BOLSA,...,"1.278,79","141,79","1.137,00",SEC. MUN. DE EDUCACAO E CULTURA,FUNDO MUNICIPAL DE EDUCACAO,MANUT.ENC. ENSINO FUNDAMENTAL - EFETIVO,EFETIVO,NaN,Lambari d'Oeste,NaN
254,202004,MT,Rio Branco,1.288952e+10,***597322**,MARIA JOSE DA ROCHA,-2,NaN,NaN,CADUN NAO BOLSA,...,"2.839,63","1.336,71","1.502,92",SECRETARIA DE ADMINISTRACAO E PLANEJAMEN,DEPARTAMENTO DE SERVICOS GERAIS,DEP DE SERVICOS GERAIS - EFETIVO,GERAL,NaN,São José dos Quatro Marcos,NaN
